# Imputation for humanskin data

## loading package

In [5]:
import scanpy as sc
import torch as th
import torch.nn as nn
from matplotlib import rcParams
from SpaMIE.create_graph import Sagegraph
from SpaMIE.spamie_main import Sagewrapper
config = {
    "font.family":'Times New Roman', 
     "font.size":20, 
    "axes.unicode_minus": False 
}
rcParams.update(config)

## Imputation

In [3]:
device = th.device('cuda:0' if th.cuda.is_available() else 'cpu')
file_fold = '/data/xiangdw/data/data/'
a = []
layers_nums = 3
for i in range(1):
    seeds = i+1
    adata_omics1 = sc.read_h5ad(file_fold + 'adata_RNA_humanskin.h5ad')
    adata_omics2 = sc.read_h5ad(file_fold + 'adata_ADT_humanskin.h5ad')

    modalities = [adata_omics1, adata_omics2]
    g_spatial_omics1, g_feature_omics1, adata_omics1, adata_omics2 = Sagegraph(modalities, device, datatype='Stereo-CITE-seq', batch=False)
    output_dir = '/data/xiangdw/data/pred result/'
    weight = [0,0,2]

    pred_name = 'humanskin_SpaMIE_'+str(layers_nums)+'_pred.csv'
    true_name = 'humanskin_SpaMIE_'+str(layers_nums)+'_truth.csv'

    in_feat = adata_omics1.obsm['feat'].shape[1]
    out_feat = adata_omics2.X.shape[1]

    model = Sagewrapper(seed=(int(seeds)), device=device, in_feat=in_feat, n_hidden=256, out_feat=out_feat, task='prediction', datatype='Stereo-CITE-seq',
                        layers_nums=int(layers_nums), weight=weight, epoch=700, res_type='res_add', activation=nn.LeakyReLU
                        , sagetype='mean', lr=4e-4, lr2 = 0.001)

    adata_omics1_pred, adata_omics2_pred, test_idx, train_idx,wt,alph  = model.fit(  adata_omics1, adata_omics2, g_spatial_omics1, g_feature_omics1, 
                                                                                  output_dir=output_dir, pred_name=pred_name, 
                                                                                    true_name=true_name, weight=True, save_csv=False)



/data/xiangdw/MODEL/SpaMIE/spamie_net.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.alpha = F.softmax(torch.squeeze(self.vu) + 1e-6)
